In [1]:
import os 
import arcpy
import pandas as pd
import matplotlib.pyplot as plt
arcpy.env.overwriteOutput = True

In [2]:
FVCOMnodes = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\fvcomenodes.shp"

shoreline_shapefile = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\shoreline\100k\lh_shore_ESRI_100k_USside.shp"

model_shoreline = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\FVCOM_shoreline.shp"

In [1]:
import os
import arcpy
import numpy as np
from sklearn.neighbors import NearestNeighbors

arcpy.env.overwriteOutput = True

FVCOMnodes = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\fvcom_shore_tmp.gdb\fvcom_nodes_coastal"
out_dir = r"D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME"
gdb = os.path.join(out_dir, "fvcom_shore_tmp.gdb")
if not arcpy.Exists(gdb):
    arcpy.management.CreateFileGDB(out_dir, os.path.basename(gdb))

nodes_m = os.path.join(gdb, "fvcom_nodes_3174")
arcpy.management.Project(FVCOMnodes, nodes_m, arcpy.SpatialReference(3174))

# NN spacing in meters
xs, ys = [], []
with arcpy.da.SearchCursor(nodes_m, ["SHAPE@XY"]) as cur:
    for (xy,) in cur:
        x, y = xy
        xs.append(x); ys.append(y)

coords = np.column_stack([xs, ys])
nbrs = NearestNeighbors(n_neighbors=2, algorithm="kd_tree").fit(coords)
dists, _ = nbrs.kneighbors(coords)
nn = dists[:, 1]

sr = arcpy.Describe(nodes_m).spatialReference
print("CRS:", sr.name, sr.factoryCode, sr.type)
print("Nodes:", coords.shape[0])
print("Min NN spacing (m):", float(nn.min()))
print("Median NN spacing (m):", float(np.median(nn)))
print("Mean NN spacing (m):", float(nn.mean()))
print("P10/P90 spacing (m):", float(np.percentile(nn, 10)), "/", float(np.percentile(nn, 90)))


CRS: NAD_1983_Great_Lakes_Basin_Albers 3174 Projected
Nodes: 4966
Min NN spacing (m): 63.94611884807578
Median NN spacing (m): 451.5751274574872
Mean NN spacing (m): 422.4495481151573
P10/P90 spacing (m): 195.12579572895385 / 557.80131050351


- for each FVCOM node (point), find the nearest location on the lake shoreline (polyline)

- Create a new “snapped-to-shore” point set (those nearest locations)

- Order those snapped points along the shoreline

- Build a polyline shoreline from the ordered snapped points

In [4]:

# ---- tune this to approximate your coastal node spacing ----
# if too many interior nodes -> reduce; if gaps -> increase
BUFFER_M = 1500

# ----------------------------
# 0) Temp GDB next to output
# ----------------------------
out_dir = os.path.dirname(model_shoreline)
gdb = os.path.join(out_dir, "fvcom_shore_tmp.gdb")
if not arcpy.Exists(gdb):
    arcpy.management.CreateFileGDB(out_dir, os.path.basename(gdb))

arcpy.env.workspace = gdb
arcpy.env.scratchWorkspace = gdb

def _fields(fc):
    return [f.name for f in arcpy.ListFields(fc)]

def _pick_join_field(field_list):
    # Common names ArcGIS uses for "input feature OID" in locate tables
    candidates = ["INPUTOID", "ORIG_FID", "OID", "OBJECTID", "FID", "IN_FID", "NEAR_FID"]
    for c in candidates:
        if c in field_list:
            return c
    # Sometimes it's something like <fcname>_OID
    for f in field_list:
        if f.upper().endswith("OID"):
            return f
    return None

# ----------------------------
# 1) Project shoreline to FVCOM nodes CRS
# ----------------------------
nodes_sr = arcpy.Describe(FVCOMnodes).spatialReference
shore_proj = os.path.join(gdb, "shore_proj")
arcpy.management.Project(shoreline_shapefile, shore_proj, nodes_sr)

# Ensure shoreline is polyline
shore_type = arcpy.Describe(shore_proj).shapeType.lower()
if shore_type == "polygon":
    shore_line = os.path.join(gdb, "shore_line")
    arcpy.management.PolygonToLine(shore_proj, shore_line)
else:
    shore_line = shore_proj

# Dissolve shoreline for cleaner routing
shore_diss = os.path.join(gdb, "shore_diss")
arcpy.management.Dissolve(shore_line, shore_diss)

# ----------------------------
# 2) Buffer shoreline and select nearshore nodes (boundary candidates)
# ----------------------------
shore_buf = os.path.join(gdb, "shore_buf")
arcpy.analysis.Buffer(shore_diss, shore_buf, f"{BUFFER_M} Meters", dissolve_option="ALL")

nodes_lyr = "nodes_lyr"
arcpy.management.MakeFeatureLayer(FVCOMnodes, nodes_lyr)
arcpy.management.SelectLayerByLocation(nodes_lyr, "INTERSECT", shore_buf)

nodes_coastal = os.path.join(gdb, "fvcom_nodes_coastal")
arcpy.management.CopyFeatures(nodes_lyr, nodes_coastal)

coastal_count = int(arcpy.management.GetCount(nodes_coastal)[0])
print(f"✅ Coastal nodes selected within {BUFFER_M} m: {coastal_count}")

# ----------------------------
# 3) Add stable source ID to coastal nodes (prevents OID confusion later)
# ----------------------------
nodes_oid = arcpy.Describe(nodes_coastal).OIDFieldName
if "SRC_OID" not in _fields(nodes_coastal):
    arcpy.management.AddField(nodes_coastal, "SRC_OID", "LONG")
    arcpy.management.CalculateField(nodes_coastal, "SRC_OID", f"!{nodes_oid}!", "PYTHON3")

# ----------------------------
# 4) NEAR to get nearest shoreline location (NEAR_X/NEAR_Y)
# ----------------------------
arcpy.analysis.Near(
    in_features=nodes_coastal,
    near_features=shore_diss,
    search_radius="",
    location="LOCATION",
    angle="NO_ANGLE",
    method="PLANAR"
)

# ----------------------------
# 5) Create snapped points at NEAR_X/NEAR_Y (carry SRC_OID)
# ----------------------------
snapped_pts = os.path.join(gdb, "fvcom_snapped_pts")
arcpy.management.XYTableToPoint(
    in_table=nodes_coastal,
    out_feature_class=snapped_pts,
    x_field="NEAR_X",
    y_field="NEAR_Y",
    coordinate_system=nodes_sr
)
# ----------------------------
# 6) Locate snapped points along the route (MEAS ordering)
# ----------------------------
loc_table = os.path.join(gdb, "snapped_loc")
arcpy.lr.LocateFeaturesAlongRoutes(
    in_features=snapped_pts,
    in_routes=shore_route,
    route_id_field="RID",
    radius_or_tolerance="1000 Meters",   # <-- bump up (robust)
    out_table=loc_table,
    out_event_properties="RID POINT MEAS",
    route_locations="FIRST"
)

print("snapped_pts count:", int(arcpy.management.GetCount(snapped_pts)[0]))
print("loc_table count:", int(arcpy.management.GetCount(loc_table)[0]))

# If loc_table is empty, stop here (that's the reason output is empty)
if int(arcpy.management.GetCount(loc_table)[0]) == 0:
    raise RuntimeError(
        "LocateFeaturesAlongRoutes returned 0 rows. "
        "This means ArcGIS couldn't locate your snapped points on the route. "
        "Common fixes: ensure shore_diss is a polyline, try MultipartToSinglepart, "
        "or increase tolerance even more."
    )

# ----------------------------
# 7) Join MEAS back to snapped points (don't pre-add MEAS; let JoinField add it)
# ----------------------------
loc_fields = [f.name for f in arcpy.ListFields(loc_table)]
print("Locate table fields:", loc_fields)

def pick_join_field(field_list):
    candidates = ["INPUTOID", "ORIG_FID", "OID", "OBJECTID", "FID", "IN_FID"]
    for c in candidates:
        if c in field_list:
            return c
    for f in field_list:
        if f.upper().endswith("OID"):
            return f
    return None

join_field_in_loc = pick_join_field(loc_fields)
if join_field_in_loc is None:
    raise RuntimeError(f"Could not find an OID-like join field in locate table: {loc_fields}")

snapped_oid = arcpy.Describe(snapped_pts).OIDFieldName
print("Using snapped OID field:", snapped_oid)
print("Using locate join field:", join_field_in_loc)

# Remove old MEAS if it exists (prevents name conflicts)
if "MEAS" in [f.name for f in arcpy.ListFields(snapped_pts)]:
    try:
        arcpy.management.DeleteField(snapped_pts, ["MEAS"])
    except:
        pass

arcpy.management.JoinField(
    in_data=snapped_pts,
    in_field=snapped_oid,
    join_table=loc_table,
    join_field=join_field_in_loc,
    fields=["MEAS", "RID"]
)

# ----------------------------
# 8) Filter only points that have MEAS (critical)
# ----------------------------
snapped_lyr = "snapped_lyr"
arcpy.management.MakeFeatureLayer(snapped_pts, snapped_lyr)

arcpy.management.SelectLayerByAttribute(snapped_lyr, "NEW_SELECTION", "MEAS IS NOT NULL")
snapped_good = os.path.join(gdb, "snapped_pts_meas_ok")
arcpy.management.CopyFeatures(snapped_lyr, snapped_good)

good_n = int(arcpy.management.GetCount(snapped_good)[0])
print("snapped points with MEAS:", good_n)

if good_n < 2:
    raise RuntimeError(
        "Fewer than 2 snapped points have MEAS, so PointsToLine cannot create a line. "
        "This means the MEAS join didn't populate correctly."
    )

# ----------------------------
# 9) Build shoreline polyline from ordered snapped points
# ----------------------------
arcpy.management.PointsToLine(
    Input_Features=snapped_good,
    Output_Feature_Class=model_shoreline,
    Line_Field="RID",
    Sort_Field="MEAS"
)

print("✅ Output shoreline count:", int(arcpy.management.GetCount(model_shoreline)[0]))
print("✅ Model shoreline:", model_shoreline)




✅ Coastal nodes selected within 1500 m: 4966
snapped_pts count: 4966
loc_table count: 4966
Locate table fields: ['OBJECTID', 'RID', 'MEAS', 'Distance', 'InstanceID', 'OID_1', 'x', 'y', 'SRC_OID', 'NEAR_FID', 'NEAR_DIST', 'NEAR_X', 'NEAR_Y']
Using snapped OID field: OBJECTID
Using locate join field: OBJECTID
snapped points with MEAS: 4966
✅ Output shoreline count: 1
✅ Model shoreline: D:\Users\abolmaal\Arcgis\NASAOceanProject\GIS_layer\Basins\FVCOME\FVCOM_shoreline.shp


In [ ]:
# find the resolution of the shoreline 
